<a href="https://colab.research.google.com/github/thirayume/poprediction/blob/main/Prediction_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **V.2 - Muangtai Purchasing Prediction by Day**

In [ ]:
# @title Define secrets

from google.colab import userdata
host = userdata.get('host')
port = userdata.get('port')
database = userdata.get('database')
user = userdata.get('user')
password = userdata.get('password')

In [ ]:
# @title Install dependencies (if need)

!pip install psycopg2 pandas pmdarima

In [ ]:
# @title Import dependencies

import psycopg2

from datetime import datetime

import numpy as np
import pandas as pd
from pandas import plotting

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns

import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected = True)

import fastai

import warnings
import os
from pathlib import Path
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.autograd import Variable

import pmdarima as pm

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from math import sqrt

# plt.style.use('fivethirtyeight')
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout = True)
plt.rc("axes", labelweight = "bold", labelsize = "large", titleweight = "bold", titlesize = 12, titlepad = 10)

In [ ]:
# @title Connect to the PostgreSQL database

conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password,
)
conn

In [ ]:
# @title Define a SQL query to select data from a "fact_picking_lists" like in a View

sql = """
      select * from view_fact_picking_lists;
      """

In [ ]:
# @title Create a cursor object to execute queries to dataframe

with conn.cursor() as cursor:
  # Execute the query
  cursor.execute(sql)
  columns = [desc[0] for desc in cursor.description]
  # Fetch all rows from the query result
  rows = cursor.fetchall()

  # Create dataframe
  df = pd.DataFrame(rows, columns=columns)

In [ ]:
# @title Close the connection
conn.close()

In [ ]:
# @title Format Date and Time and Index

df['podatetime'] = pd.to_datetime( df['dim_pickinglists__document_date']  + ' ' + df['dim_pickinglists__document_time'], infer_datetime_format=True)
df['dim_pickinglists__document_date'] = pd.to_datetime(df['dim_pickinglists__document_date'], infer_datetime_format = True)
df['podatetime'] = pd.to_datetime(df['podatetime'], infer_datetime_format = True)

In [ ]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# Fill NaN with False for this column
df['dim_families__is_alcoholic'] = df['dim_families__is_alcoholic'].notna()

In [ ]:
# Clean all null value
df1 = df.copy()
df1 = df1.drop(['liters'], axis=1)
df1.isnull().sum()

In [ ]:
# Drop some data
mindate = datetime.datetime(2023, 4, 24)

df1 = df1[df1.dim_pickinglists__document_date > mindate]

In [ ]:
sku_ids = df1['dim_stock_keeping_units__sku_id'].unique()
print(sorted(sku_ids))

In [ ]:
# @title Pivot SKU Sales by Month
pd.set_option("display.float_format", "{:,.0f}".format)

df1['dim_pickinglists__document_date'] = pd.to_datetime(df1['dim_pickinglists__document_date'], infer_datetime_format = True)

pivot_df1 = df1.pivot_table(
    values='fact_picking_lists__quantity',
    index=['dim_stock_keeping_units__sku_id'],
    columns=['dim_pickinglists__document_date'],
    aggfunc='sum',
    margins = True,
    fill_value = '0'
)
pivot_df1 = pivot_df1.sort_values(by=['All'], ascending=False)
pivot_df1

In [ ]:
pivot_df1.T

In [ ]:
sorted_sku = []
for sku, qty in pivot_df1.iterrows():
  sorted_sku.append(sku)
sorted_sku.pop(0)

In [ ]:
def plot_by_SKUs(sku_df, pivot_df):
  for i in range(len(sku_df)):
    plt.figure(figsize=(6,3))

    title = 'SKU: ' + sku_df[i]
    xlabel = ''
    ylabel = ''

    my_df = pivot_df.T[sku_df[i]]
    my_df.drop(my_df.tail(1).index,inplace=True)
    my_df.astype(float).plot()

    plt.title(label=title)
    plt.autoscale(axis='x', tight=True)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig('SKU/' + sku_df[i] + '.png', bbox_inches='tight')
    plt.show()

In [ ]:
sample_sku_df = sorted_sku
plot_by_SKUs(sample_sku_df, pivot_df1)

sample_sku_df = sorted_sku[0:1]

In [ ]:
def mape(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
sample_df1 = pivot_df1.T[sample_sku_df].reset_index()
sample_df1.drop(sample_df1.tail(1).index, inplace=True)
sample_df1.columns = ['month', 'qty']
sample_df1['month'] = pd.to_datetime(sample_df1['month'], infer_datetime_format = True)
sample_df1 = sample_df1.set_index(['month'])
sample_df1

In [ ]:
sample_df1.info()

In [ ]:
sample_df1.isnull().sum()

In [ ]:
plt.figure(figsize = (15, 7))
plt.title("Number of Purchase by Date")
plt.xlabel('Date')
plt.ylabel('QTY')
plt.plot(sample_df1)
plt.show()

### <center> Rolling Statistics

A rolling average is a great way to visualize how the dataset is trending. As the dataset provides counts by month, a window size of 12 will give the annual rolling average.

this plot include the rolling standard deviation to see how much the data varies from the rolling average.

In [ ]:
#Determine rolling statistics
sample_df1["rolling_avg"] = sample_df1["qty"].rolling(window = 12).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
sample_df1["rolling_std"] = sample_df1["qty"].rolling(window = 12).std()

#Plot rolling statistics
plt.figure(figsize = (15, 7))
plt.plot(sample_df1["qty"], color = '#379BDB', label = 'Original')
plt.plot(sample_df1["rolling_avg"], color = '#D22A0D', label = 'Rolling Mean')
plt.plot(sample_df1["rolling_std"], color = '#142039', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Standard Deviation')
plt.savefig(sample_sku_df[0] + '_Rolling.png', bbox_inches='tight')
plt.show(block = False)

### <center>Augmented Dickey–Fuller Test</center>

The Augmented Dickey-Fuller Test is used to determine if time-series data is stationary or not. Similar to a t-test, let set a significance level before the test and make conclusions on the hypothesis based on the resulting p-value.

<B>Null Hypothesis:</B> The data is not stationary.

<B>Alternative Hypothesis:</B> The data is stationary.

For the data to be stationary (ie. reject the null hypothesis), the ADF test should have:

- p-value <= significance level (0.01, 0.05, 0.10, etc.)

If the p-value is greater than the significance level then we can say that it is likely that the data is not stationary.

In [ ]:
# Prepare dataset
sample_df1 = pivot_df1.T[sample_sku_df].reset_index()
sample_df1.drop(sample_df1.tail(1).index, inplace=True)
sample_df1.columns = ['month', 'qty']
sample_df1['month'] = pd.to_datetime(sample_df1['month'], infer_datetime_format = True)
sample_df1 = sample_df1.set_index(['month'])

### <center>SARIMA Model Selection</center>

Now let's try the same strategy as above, except let's use a SARIMA model so that can be account for seasonality.

In [ ]:
result = seasonal_decompose(sample_df1, model='additive', extrapolate_trend='freq', period=12)
fig = plt.figure()
fig = result.plot()
fig.savefig(sample_sku_df[0] + '_seasonal_decompose.png', bbox_inches='tight')

In [ ]:
#Augmented Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(sample_df1['qty'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value

print(dfoutput)

In [ ]:
result = adfuller(sample_df1.qty.dropna())
print(f'ADF Statistics:{result[0]}')
print(f'p-value:{result[1]}')

The p-value is higher than 0.05. This means that the time serie is non stationary with a confidence of 95%. Then check if with a one step differentiation, the time serie become stationary (in terms of a trendless time series).

In [ ]:
result2 = adfuller(sample_df1.qty.diff().dropna())
print(f'ADF Statistics:{result2[0]}')
print(f'p-value:{result2[1]}')

In [ ]:
# @title ACF and PACF
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (8, 8))

plot_acf(sample_df1, lags = 12, zero = False, ax = ax1)
plot_pacf(sample_df1, lags = 12, zero = False, ax = ax2)
plt.savefig(sample_sku_df[0] + '_acf_pacf.png', bbox_inches='tight')
plt.show()

In [ ]:
# Seasonal - fit stepwise auto-ARIMA
SARIMA_model = pm.auto_arima(sample_df1, start_p = 1, start_q = 1,
                        max_p = 3, max_q = 3,
                        m = 12, # 12 is the frequncy of the cycle
                        seasonal = True, # set to seasonal
                        d = 1,
                        D = 1, # order of the seasonal differencing
                        trace = True,
                        error_action = 'ignore',
                        start_P = 1, start_Q = 1,
                        max_P = 2, max_Q = 2,
                        information_criterion = 'aic',
                        stepwise = True)

### SARIMA (3,1,0)(2,1,0)[12]   : AIC=5488.455 - intercept   : AIC=5490.458

In [ ]:
model1 = SARIMAX(sample_df1.astype(float), order = (3, 1, 0), seasonal_order = (2, 1, 0, 12))
SARIMA_model1 = model1.fit()

In [ ]:
SARIMA_model1.summary()

In [ ]:
# @title Create the 4 diagostics plots
SARIMA_model1.plot_diagnostics(figsize = (8, 8))
plt.savefig(sample_sku_df[0] + '_model1_diag.png', bbox_inches='tight')
plt.show()

In [ ]:
# All the 4 plots indicates a good fit of the SARIMA model on the given time serie.
# Create a vector that will host the predictions
prediction1 = SARIMA_model1.get_prediction(start = -12)
mean_prediction1 = prediction1.predicted_mean
mean_prediction1 = mean_prediction1.rename("prediction1")

In [ ]:
# Get the confidence intervals from the sarima prediction
confi_int_p1 = prediction1.conf_int()
lower_limits_p1 = confi_int_p1.iloc[:, 0]
upper_limits_p1 = confi_int_p1.iloc[:, 1]

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df1[-24:].index, sample_df1[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction1[-24:].index, mean_prediction1[-24:].values, label = 'Prediction', color = "green", marker = "o")
plt.fill_between(mean_prediction1[-24:].index, lower_limits_p1, upper_limits_p1, alpha = 0.1, color = "green")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.savefig(sample_sku_df[0] + '_model1_predict.png', bbox_inches='tight')
plt.show()

In [ ]:
mape_sarima1 = mape(sample_df1.iloc[-12:, 0], mean_prediction1)
print(f"MAPE OF LSTM MODEL : {mape_sarima1:.2f} %")

In [ ]:
rmse_sarima1 = sqrt(mean_squared_error(sample_df1[-12:].values, mean_prediction1.values))
print(f"RMSE OF LSTM MODEL : {rmse_sarima1:.2f}")

### SARIMA(2,1,0)(2,1,0)[12] : AIC=5494.111

In [ ]:
model2 = SARIMAX(sample_df1.astype(float), order = (2, 1, 0), seasonal_order = (2, 1, 0, 12))
SARIMA_model2 = model2.fit()

In [ ]:
SARIMA_model2.summary()

In [ ]:
# @title Create the 4 diagostics plots
SARIMA_model2.plot_diagnostics(figsize = (8, 8))
plt.savefig(sample_sku_df[0] + '_model2_diag.png', bbox_inches='tight')
plt.show()

In [ ]:
# All the 4 plots indicates a good fit of the SARIMA model on the given time serie.
# Create a vector that will host the predictions
prediction2 = SARIMA_model2.get_prediction(start = -12)
mean_prediction2 = prediction2.predicted_mean
mean_prediction2 = mean_prediction2.rename("prediction2")

In [ ]:
# Get the confidence intervals from the sarima prediction
confi_int_p2 = prediction2.conf_int()
lower_limits_p2 = confi_int_p2.iloc[:, 0]
upper_limits_p2 = confi_int_p2.iloc[:, 1]

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df1[-24:].index, sample_df1[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction2[-24:].index, mean_prediction2[-24:].values, label = 'Prediction 2', color = "orange", marker = "*")
plt.fill_between(mean_prediction2[-24:].index, lower_limits_p2, upper_limits_p2, alpha = 0.1, color = "orange")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.savefig(sample_sku_df[0] + '_model2_predict.png', bbox_inches='tight')
plt.show()

In [ ]:
mape_sarima2 = mape(sample_df1.iloc[-12:, 0], mean_prediction2)
print(f"MAPE OF LSTM MODEL : {mape_sarima2:.2f} %")

In [ ]:
rmse_sarima2 = sqrt(mean_squared_error(sample_df1[-12:].values, mean_prediction2.values))
print(f"RMSE OF LSTM MODEL : {rmse_sarima2:.2f}")

### SARIMA(3,1,0)(1,1,0)[12] : AIC=5512.435

In [ ]:
model3 = SARIMAX(sample_df1.astype(float), order = (3, 1, 0), seasonal_order = (1, 1, 0, 12))
SARIMA_model3 = model3.fit()

In [ ]:
SARIMA_model3.summary()

In [ ]:
# @title Create the 4 diagostics plots
SARIMA_model3.plot_diagnostics(figsize = (8, 8))
plt.savefig(sample_sku_df[0] + '_model3_diag.png', bbox_inches='tight')
plt.show()

In [ ]:
# All the 4 plots indicates a good fit of the SARIMA model on the given time serie.
# Create a vector that will host the predictions
prediction3 = SARIMA_model3.get_prediction(start = -12)
mean_prediction3 = prediction3.predicted_mean
mean_prediction3 = mean_prediction3.rename("prediction3")

In [ ]:
# Get the confidence intervals from the sarima prediction
confi_int_p3 = prediction3.conf_int()
lower_limits_p3 = confi_int_p3.iloc[:, 0]
upper_limits_p3 = confi_int_p3.iloc[:, 1]

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df1[-24:].index, sample_df1[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction3[-24:].index, mean_prediction3[-24:].values, label = 'Prediction 3', color = "indigo", marker = "x")
plt.fill_between(mean_prediction3[-24:].index, lower_limits_p3, upper_limits_p3, alpha = 0.1, color = "indigo")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.savefig(sample_sku_df[0] + '_model3_predict.png', bbox_inches='tight')
plt.show()

In [ ]:
mape_sarima3 = mape(sample_df1.iloc[-12:, 0], mean_prediction3)
print(f"MAPE OF LSTM MODEL : {mape_sarima3:.2f} %")

In [ ]:
rmse_sarima3 = sqrt(mean_squared_error(sample_df1[-12:].values, mean_prediction3.values))
print(f"RMSE OF LSTM MODEL : {rmse_sarima3:.2f}")

# Results Sumamry

In [ ]:
plt.figure(figsize = (14, 5))
plt.title("Purchase prediction by SARIMA", fontsize = 25)

plt.plot(sample_df1[-24:].index, sample_df1[-24:].values, label = 'Actual values', color = "blue", marker = "o")

plt.plot(mean_prediction1[-24:].index, mean_prediction1[-24:].values, label = 'Prediction 1', color = "green", marker = "o")
plt.fill_between(mean_prediction1[-24:].index, lower_limits_p1, upper_limits_p1, alpha = 0.1, color = "green")

plt.plot(mean_prediction2[-24:].index, mean_prediction2[-24:].values, label = 'Prediction 2', color = "orange", marker = "*")
plt.fill_between(mean_prediction2[-24:].index, lower_limits_p2, upper_limits_p2, alpha = 0.1, color = "orange")

plt.plot(mean_prediction3[-24:].index, mean_prediction3[-24:].values, label = 'Prediction 3', color = "indigo", marker = "x")
plt.fill_between(mean_prediction3[-24:].index, lower_limits_p3, upper_limits_p3, alpha = 0.1, color = "indigo")

plt.legend(fontsize = 12, fancybox = True, shadow = True, frameon = True)
plt.ylabel('QTY', fontsize = 15)
plt.savefig(sample_sku_df[0] + '_summary.png', bbox_inches='tight')
plt.show()

สรุปได้ว่าควรใช้
Best model:

1.   SARIMA(3,1,0)(2,1,0) [12]
2.   SARIMA(2,1,0)(2,1,0) [12]
3.   SARIMA(3,1,0)(1,1,0) [12]

Model ที่ 1-2 ดีกว่า และดีกว่า การใช้ datapoint รายเดือน